In [93]:
## import all necessary library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [94]:
# read loan.csv file from 
loan=pd.read_csv("/Users/shekher.r/upgrade/python/Juune-ML53/Lending_club_case_study/ravi/loan.csv")

In [87]:
## taking some insights of records
loan.head()
loan.shape
loan.info(verbose=True, show_counts=True)
loan.loan_status.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              39717 non-null  int64  
 1    member_id                       39717 non-null  int64  
 2    loan_amnt                       39717 non-null  int64  
 3    funded_amnt                     39717 non-null  int64  
 4    funded_amnt_inv                 39717 non-null  float64
 5    term                            39717 non-null  object 
 6    int_rate                        39717 non-null  object 
 7    installment                     39717 non-null  float64
 8    grade                           39717 non-null  object 
 9    sub_grade                       39717 non-null  object 
 10   emp_title                       37258 non-null  object 
 11   emp_length                      38642 non-null  object 
 12   home_ownership  

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [88]:
## filtering out records which are Fully Paid or Charged Off as we cant say anything about loan which are currently running
loan_redord_anyls=loan[(loan.loan_status=='Fully Paid')|(loan.loan_status=='Charged Off')]

In [89]:
## comparing shapes of records after removing Current loan status records
print(loan.shape)
print(loan_redord_anyls.shape)

(39717, 111)
(38577, 111)


In [96]:
#loan_redord_anyls.info(verbose=True, show_counts=True)
#Remove columns that have 90% null values
drop_columns=loan_redord_anyls.columns[(loan_redord_anyls.isnull().sum()/len(loan_redord_anyls.index))>.9]
loan_redord_anyls_1=loan_redord_anyls.drop(drop_columns,axis=1)
print(loan_redord_anyls_1.isnull().sum())
loan_redord_anyls_1.info()


id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2386
emp_length                     1033
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12527
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

In [97]:
## remove columns id,member_id,desc
loan_redord_anyls_1.head(2)
loan_redord_anyls_2=loan_redord_anyls_1.drop(["id","member_id","desc"],axis=1)
print(loan.shape)
print(loan_redord_anyls_2.shape)


(39717, 111)
(38577, 52)


In [113]:
## remove records that have more than 5 columns missing
pd.set_option('display.max_columns', None)
loan_redord_anyls_3=loan_redord_anyls_2[~(loan_redord_anyls_2.isnull().sum(axis=1)>5)]
loan_redord_anyls_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38575 entries, 0 to 39716
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   38575 non-null  int64  
 1   funded_amnt                 38575 non-null  int64  
 2   funded_amnt_inv             38575 non-null  float64
 3   term                        38575 non-null  object 
 4   int_rate                    38575 non-null  object 
 5   installment                 38575 non-null  float64
 6   grade                       38575 non-null  object 
 7   sub_grade                   38575 non-null  object 
 8   emp_title                   36191 non-null  object 
 9   emp_length                  37542 non-null  object 
 10  home_ownership              38575 non-null  object 
 11  annual_inc                  38575 non-null  float64
 12  verification_status         38575 non-null  object 
 13  issue_d                     385

In [117]:
## change data type of column
loan_redord_anyls_3['term'].head()
loan_redord_anyls_3['term']=loan_redord_anyls_3['term'].apply(lambda x:int(x.replace(' months','')))
loan_redord_anyls_3['term']=loan_redord_anyls_3['term'].apply(lambda x:int(x.replace(' months','')))

/var/folders/8c/82t91qxx77x0n9h3xtznbr8r0000gq/T/ipykernel_84649/3912267848.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_redord_anyls_3['term']=loan_redord_anyls_3['term'].apply(lambda x:int(x.replace(' months','')))


In [118]:
loan_redord_anyls_3['term'].head()

0    36
1    60
2    36
3    36
5    36
Name: term, dtype: int64